In [36]:
%pip install PyMuPDF

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [37]:
import fitz  # PyMuPDF
import pandas as pd
import re

# Chemin vers votre fichier PDF
pdf_path = "listing_depart.pdf"

# Ouvrir le document PDF
pdf_document = fitz.open(pdf_path)

# Nombre de pages dans le PDF
num_pages = pdf_document.page_count
print(f"Nombre de pages : {num_pages}")

# Motif à rechercher
titre_pattern = r"Epreuve n°\d+ : .+"
exclusion_pattern = "Num\nCouple\nClub / Race Équidé"

# Créer une liste pour stocker les données extraites
data = []

# Extraire le texte de chaque page après le titre spécifié
for page_num in range(num_pages):
    page = pdf_document.load_page(page_num)
    text = page.get_text()
    
    # Supprimer les lignes contenant le motif d'exclusion
    text = text.replace(exclusion_pattern, "")
    
    # Rechercher le motif d'épreuve
    matches = re.finditer(titre_pattern, text)
    
    for match in matches:
        epreuve = match.group()
        start_index = match.end()
        
        # Extraire le texte après le motif de l'épreuve
        remaining_text = text[start_index:]
        
        # Séparer les lignes de cavaliers
        lines = remaining_text.split('\n')
        
        for i in range(0, len(lines) - 5, 6):
            try:
                num = lines[i].strip()
                cavalier = lines[i + 1].strip()
                team = lines[i + 2].strip()
                coach = lines[i + 3].strip()
                club = lines[i + 4].strip()
                race = lines[i + 5].strip()
                
                # Ajouter les données au tableau
                data.append([epreuve, num, cavalier, team, coach, club, race])
            except IndexError:
                continue

# Créer un DataFrame pandas
columns = ['Num_Epreuve', 'Cavalier', 'Team', 'Coach', 'Club', 'Race', 'Num']
df = pd.DataFrame(data, columns=columns)

# Decaller les éléments de la  colonne Race d'une ligne vers le bas
df["Num"]=df["Num"].shift(1)
# Supprimer toutes les ligne scontenant le most page
df = df[~df['Coach'].str.contains('page')]

Nombre de pages : 6


In [38]:
# Convertir explicitement la colonne en chaînes de caractères
df['Num_Epreuve'] = df['Num_Epreuve'].astype(str)

# Splitter la colonne Num_Epreuve suivant le motif ':'
df[['Num_Epreuve', 'Nom_Epreuve']] = df['Num_Epreuve'].str.split(' : ', expand=True)


In [43]:
df.head(20)

,Num_Epreuve,Cavalier,Team,Coach,Club,Race,Num,Description
1,Epreuve n°1,LANA RONCE,RIVAL DE KERMOAL,Coach : ANNE LAURE LE SAINT (5699354G),BREST EQUITATION,PONEY FRANCAIS DE SELLE,22,Hunter Club 2 Mixte Mania Imp 70 cm
2,Epreuve n°1,ERIC SASSOWER,BELIZON D'HELLEZ,Coach : HAUDE GELEBART (0028983K),ECURIE GELEBART EARL,SELLE FRANCAIS,14,Hunter Club 2 Mixte Mania Imp 70 cm
3,Epreuve n°1,MAIWEN RABIN,TWIX DU VENEC,Coach : FRANCOISE JEZEQUEL (5454636E),PONEY CLUB DE LANVERON,ORIGINE CONSTATEE,15,Hunter Club 2 Mixte Mania Imp 70 cm
4,Epreuve n°1,CELINE CHAPIN,SILVER DE COQUERIE,Coach : ANNE LAURE LE SAINT (5699354G),BREST EQUITATION,SELLE FRANCAIS SECTION B,18,Hunter Club 2 Mixte Mania Imp 70 cm
5,Epreuve n°1,MARION ROUYER,CARWYN DE LANVERON,Coach : FRANCOISE JEZEQUEL (5454636E),PONEY CLUB DE LANVERON,WELSH COB,17,Hunter Club 2 Mixte Mania Imp 70 cm
6,Epreuve n°1,MARION ROUYER,EMRISH TOUL AR C'HOAT,Coach : FRANCOISE JEZEQUEL (5454636E),PONEY CLUB DE LANVERON,CONNEMARA,8,Hunter Club 2 Mixte Mania Imp 70 cm
7,Epreuve n°1,CHARLINE MAGUEUR,RADAR,Coach : HAUDE GELEBART (0028983K),ECURIE GELEBART EARL,SELLE FRANCAIS SECTION B,13,Hunter Club 2 Mixte Mania Imp 70 cm
8,Epreuve n°1,SWANN POSTIC,PLOWNE DE VIEVE,Coach : HAUDE GELEBART (0028983K),ECURIE GELEBART EARL,SELLE FRANCAIS SECTION A,12,Hunter Club 2 Mixte Mania Imp 70 cm
9,Epreuve n°1,YAELLE L HERMITTE,CALOUBET DU ROJO,Coach : VERONIQUE COLCANAP (0403375Z),E E DE POMMERIT JAUDY,SELLE FRANCAIS,2,Hunter Club 2 Mixte Mania Imp 70 cm
10,Epreuve n°1,MATHILDE D ARRIGO,ELLA DU VERT,Coach : HAUDE GELEBART (0028983K),ECURIE GELEBART EARL,SELLE FRANCAIS,20,Hunter Club 2 Mixte Mania Imp 70 cm


In [40]:
# Exportar el dataframe a un archivo CSV
df.to_csv('liste_depart.csv', index=False)